In [1]:
import ray
from ray import tune
import time

In [2]:
f=open('head_node_info','r+')
buf=f.readlines()
buf=buf[0].strip().split()
address=buf[0]
redis_pass=buf[1]
f.close()

In [3]:
ray.init(address=address,_redis_password=redis_pass)

{'node_ip_address': '10.109.69.68',
 'raylet_ip_address': '10.109.69.68',
 'redis_address': None,
 'object_store_address': '/tmp/ray/session_2022-03-19_16-24-56_728068_151182/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-03-19_16-24-56_728068_151182/sockets/raylet',
 'webui_url': '10.109.69.62:9122',
 'session_dir': '/tmp/ray/session_2022-03-19_16-24-56_728068_151182',
 'metrics_export_port': 60870,
 'gcs_address': '10.109.69.62:9121',
 'address': '10.109.69.62:9121',
 'node_id': '461ba8f0acc58a687a9d1427c28dd33f3270711fac1faa74888fa6c6'}

In [4]:
def objective(x, a, b):
    return a * (x ** 0.5) + b

In [5]:
def trainable (config):
    a,b = config['a'],config['b']
    # call the evalution funtion on diferent values of x
    for x in range(20):
        score = objective (x, a, b)
        # Send results to Tune
        tune.report(score = score)  

analysis = tune.run(trainable,
                    num_samples=100,
                    resources_per_trial={'cpu':1},
                    max_concurrent_trials=6,
                    config={
                        "a": tune.uniform(0, 20),
                        "b": tune.uniform(0,20)} )

In [6]:
analysis = tune.run(
    trainable,
    num_samples=100,
    stop={"training_iteration": 20},
    resources_per_trial={"cpu":1},
    config={"a": tune.uniform(0, 20),
            "b": tune.uniform(0,20)},
    max_concurrent_trials=8 )

2022-03-19 17:02:51,951	WARNING function_runner.py:562 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,a,b
trainable_444b6_00000,PENDING,,3.66846,9.73051
trainable_444b6_00001,PENDING,,19.8974,1.79102
trainable_444b6_00002,PENDING,,12.9891,14.5258
trainable_444b6_00003,PENDING,,15.6144,0.631268
trainable_444b6_00004,PENDING,,11.0928,6.69002
trainable_444b6_00005,PENDING,,1.58748,0.0317774
trainable_444b6_00006,PENDING,,2.4623,17.9851
trainable_444b6_00007,PENDING,,5.6247,1.82188


Result for trainable_444b6_00002:
  date: 2022-03-19_17-02-54
  done: false
  experiment_id: b5dcf38aeee74c42a5ccc06e542bc8ac
  hostname: cn513-12-r
  iterations_since_restore: 1
  node_ip: 10.109.69.68
  pid: 107412
  score: 14.525804379746784
  time_since_restore: 0.00011396408081054688
  time_this_iter_s: 0.00011396408081054688
  time_total_s: 0.00011396408081054688
  timestamp: 1647698574
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 444b6_00002
  
Result for trainable_444b6_00001:
  date: 2022-03-19_17-02-54
  done: false
  experiment_id: 46cea28b8bbc4b848015f78cc4b00159
  hostname: cn513-12-r
  iterations_since_restore: 1
  node_ip: 10.109.69.68
  pid: 107413
  score: 1.791021021257837
  time_since_restore: 0.0001804828643798828
  time_this_iter_s: 0.0001804828643798828
  time_total_s: 0.0001804828643798828
  timestamp: 1647698574
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 444b6_00001
  
Result for trainable_444b6_00000:
  date: 2022-03-19

Trial name,status,loc,a,b,iter,total time (s),score
trainable_444b6_00004,RUNNING,10.109.69.62:166915,11.0928,6.69002,,,
trainable_444b6_00005,RUNNING,10.109.69.62:166914,1.58748,0.0317774,,,
trainable_444b6_00006,RUNNING,10.109.69.68:107414,2.4623,17.9851,,,
trainable_444b6_00007,RUNNING,10.109.69.68:107589,5.6247,1.82188,,,
trainable_444b6_00008,PENDING,,13.6542,5.19137,,,
trainable_444b6_00009,PENDING,,8.83647,1.22885,,,
trainable_444b6_00010,PENDING,,15.9153,18.8983,,,
trainable_444b6_00011,PENDING,,9.01409,9.55909,,,
trainable_444b6_00000,TERMINATED,10.109.69.68:107411,3.66846,9.73051,20,0.234812,25.721
trainable_444b6_00001,TERMINATED,10.109.69.68:107413,19.8974,1.79102,20,0.157242,88.5219


Result for trainable_444b6_00006:
  date: 2022-03-19_17-02-57
  done: false
  experiment_id: 89609dfc5db34f5487054d2f44d4728e
  hostname: cn513-12-r
  iterations_since_restore: 1
  node_ip: 10.109.69.68
  pid: 107414
  score: 17.985143199488654
  time_since_restore: 0.012913227081298828
  time_this_iter_s: 0.012913227081298828
  time_total_s: 0.012913227081298828
  timestamp: 1647698577
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 444b6_00006
  
Result for trainable_444b6_00004:
  date: 2022-03-19_17-02-57
  done: false
  experiment_id: 05c7d29f8cd3493f835bb466696a1db2
  hostname: cn513-11-r
  iterations_since_restore: 1
  node_ip: 10.109.69.62
  pid: 166915
  score: 6.69001571407226
  time_since_restore: 0.00018024444580078125
  time_this_iter_s: 0.00018024444580078125
  time_total_s: 0.00018024444580078125
  timestamp: 1647698577
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 444b6_00004
  
Result for trainable_444b6_00007:
  date: 2022-03-19_17-

...

Result for trainable_444b6_00092:
  date: 2022-03-19_17-04-13
  done: false
  experiment_id: 720d8df22f5b41b09a9720b95f2bdcfe
  hostname: cn513-12-r
  iterations_since_restore: 1
  node_ip: 10.109.69.68
  pid: 109827
  score: 12.855888070262324
  time_since_restore: 0.0001971721649169922
  time_this_iter_s: 0.0001971721649169922
  time_total_s: 0.0001971721649169922
  timestamp: 1647698653
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 444b6_00092
  
Result for trainable_444b6_00092:
  date: 2022-03-19_17-04-13
  done: true
  experiment_id: 720d8df22f5b41b09a9720b95f2bdcfe
  hostname: cn513-12-r
  iterations_since_restore: 20
  node_ip: 10.109.69.68
  pid: 109827
  score: 54.600992551315805
  time_since_restore: 0.07843732833862305
  time_this_iter_s: 0.004464387893676758
  time_total_s: 0.07843732833862305
  timestamp: 1647698653
  timesteps_since_restore: 0
  training_iteration: 20
  trial_id: 444b6_00092
  
Result for trainable_444b6_00093:
  date: 2022-03-19_17-04

Trial name,status,loc,a,b,iter,total time (s),score
trainable_444b6_00098,RUNNING,10.109.69.62:168032,9.13719,9.55513,,,
trainable_444b6_00099,PENDING,,4.3388,19.2547,,,
trainable_444b6_00000,TERMINATED,10.109.69.68:107411,3.66846,9.73051,20,0.234812,25.721
trainable_444b6_00001,TERMINATED,10.109.69.68:107413,19.8974,1.79102,20,0.157242,88.5219
trainable_444b6_00002,TERMINATED,10.109.69.68:107412,12.9891,14.5258,20,0.26546,71.1439
trainable_444b6_00003,TERMINATED,10.109.69.68:107495,15.6144,0.631268,20,0.094759,68.6928
trainable_444b6_00004,TERMINATED,10.109.69.62:166915,11.0928,6.69002,20,2.00702,55.0423
trainable_444b6_00005,TERMINATED,10.109.69.62:166914,1.58748,0.0317774,20,1.68841,6.95145
trainable_444b6_00006,TERMINATED,10.109.69.68:107414,2.4623,17.9851,20,1.34115,28.718
trainable_444b6_00007,TERMINATED,10.109.69.68:107589,5.6247,1.82188,20,0.34793,26.3394


Result for trainable_444b6_00098:
  date: 2022-03-19_17-04-19
  done: false
  experiment_id: 82eb32e01da9439c8abae8457286b548
  hostname: cn513-11-r
  iterations_since_restore: 1
  node_ip: 10.109.69.62
  pid: 168032
  score: 9.555126472876957
  time_since_restore: 0.00011301040649414062
  time_this_iter_s: 0.00011301040649414062
  time_total_s: 0.00011301040649414062
  timestamp: 1647698659
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 444b6_00098
  
Result for trainable_444b6_00098:
  date: 2022-03-19_17-04-19
  done: true
  experiment_id: 82eb32e01da9439c8abae8457286b548
  hostname: cn513-11-r
  iterations_since_restore: 20
  node_ip: 10.109.69.62
  pid: 168032
  score: 49.383234724219
  time_since_restore: 0.0880880355834961
  time_this_iter_s: 0.003330707550048828
  time_total_s: 0.0880880355834961
  timestamp: 1647698659
  timesteps_since_restore: 0
  training_iteration: 20
  trial_id: 444b6_00098
  
Result for trainable_444b6_00099:
  date: 2022-03-19_17-04-19

Trial name,status,loc,a,b,iter,total time (s),score
trainable_444b6_00000,TERMINATED,10.109.69.68:107411,3.66846,9.73051,20,0.234812,25.721
trainable_444b6_00001,TERMINATED,10.109.69.68:107413,19.8974,1.79102,20,0.157242,88.5219
trainable_444b6_00002,TERMINATED,10.109.69.68:107412,12.9891,14.5258,20,0.26546,71.1439
trainable_444b6_00003,TERMINATED,10.109.69.68:107495,15.6144,0.631268,20,0.094759,68.6928
trainable_444b6_00004,TERMINATED,10.109.69.62:166915,11.0928,6.69002,20,2.00702,55.0423
trainable_444b6_00005,TERMINATED,10.109.69.62:166914,1.58748,0.0317774,20,1.68841,6.95145
trainable_444b6_00006,TERMINATED,10.109.69.68:107414,2.4623,17.9851,20,1.34115,28.718
trainable_444b6_00007,TERMINATED,10.109.69.68:107589,5.6247,1.82188,20,0.34793,26.3394
trainable_444b6_00008,TERMINATED,10.109.69.68:107582,13.6542,5.19137,20,0.0768204,64.7086
trainable_444b6_00009,TERMINATED,10.109.69.68:107583,8.83647,1.22885,20,0.0906367,39.7461


2022-03-19 17:04:20,093	INFO tune.py:639 -- Total run time: 88.15 seconds (87.86 seconds for the tuning loop).


In [7]:
print("Best config is:", analysis.get_best_config(metric="score", mode="max"))

Best config is: {'a': 19.995269998226263, 'b': 12.83175450986751}


In [14]:
analysis.get_all_configs

trainable_444b6_00032